## Esse notebook tem o objetivo de documentar a evolução do projeto

In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import json
from pyspark.sql.functions import mean, min, max

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext
sqlContext = pyspark.SQLContext(sc)

df= spark.read.json('../yelp_academic_dataset_review.json')
df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|ujmEBvifdJM6h6RLv...|   0|2013-05-07 04:34:36|    1|Q1sbwvVQXV2734tPg...|  1.0|Total bill for th...|     6|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|   0|2017-01-14 21:30:33|    0|GJXCdrto3ASJOqKeV...|  5.0|I *adore* Travis ...|     0|yXQM5uF2jS6es16SJ...|
|WTqjgwHlXbSFevF32...|   0|2016-11-09 20:09:03|    0|2TzJjDVDEuAW6MR5V...|  5.0|I have to say tha...|     3|n6-Gk65cPZL6Uz8qR...|
|ikCg8xy5JIg_NGPx-...|   0|2018-01-09 20:56:38|    0|yi0R0Ugj_xUx_Nek0...|  5.0|Went in for a lun...|     0|dacAIZ6fTM6mqwW5u...|
|b1b1eb3uo-w561D0Z...|   0|2018-01-30 23:07:38|    0|11a8sVPMUFtaC7_AB...|  1.0|Today was 

In [2]:
print((df.count(), len(df.columns)))

(6685900, 9)


In [3]:
df.first()

Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA')

In [4]:
df.describe().show()

+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|summary|         business_id|              cool|               date|              funny|           review_id|             stars|                                text|            useful|             user_id|
+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|  count|             6685900|           6685900|            6685900|            6685900|             6685900|           6685900|                             6685900|           6685900|             6685900|
|   mean|                null|0.5787708461089756|               null|0.48276671801851656|                null|3.7161993149762935|                                null|1.3541

In [5]:
gr = df.agg(countDistinct("business_id"))
gr.show()

+---------------------------+
|count(DISTINCT business_id)|
+---------------------------+
|                     192606|
+---------------------------+



## Explorando as possibilidades com um buisness especifico

In [6]:
def get_buissness(biz_id):   
    df_b = df.filter(df['business_id'] == biz_id).select('stars','date','text','useful') # Separa o df com somente o buisness de intresse

    df_size = df_b.count() # Conta o numero de reviews
    avg_rating = (df_b.agg({"stars": "avg"})).first()[0] # Faz a media de reviews
    std = (df_b.agg({"stars": "stddev"})).first()[0] # Calculda o desvio padrao
    worst = df_b.orderBy(asc("stars"),desc("useful")).limit(1).first() # Seleciona a review com nota mais baixa e mais marcada como 'useful'
    
    print("Quantidade de Reviews: ", df_size)
    print("Media das Avaliacoes: ", avg_rating)
    print("Desvio padrao: ", std)
    print("\nPior review: ", worst['stars'], "estrelas em ", worst['date'])
    print("Comentario da pior review: ", worst['text'])

In [16]:
#biz_id = 'Gyrez6K8f1AyR7dzW9fvAw'
biz_id = 'FKQUxOfW_W5R4OTut4G1Vw'
#NZnhc2sEQy3RmzKTZnqtwQ

get_buissness(biz_id)

Quantidade de Reviews:  39
Media das Avaliacoes:  4.230769230769231
Desvio padrao:  0.9586596581065523

Pior review:  1.0 estrelas em  2011-02-19 20:35:31
Comentario da pior review:  I found a short black curly hair in my basket of fries... and it was definitely not mine.

Needless to say, this completely ruined my meal. Thanks to Peppi's, I won't be eating anything that isn't ready-to-eat as is (like fruit) for the next few days.

Not only does the shop LOOK grimey, it IS grimey... Completely disgusted.


## Pegando informacoes sobre o buisness na sua pagina no Yelp

In [17]:
from bs4 import BeautifulSoup
import requests
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [18]:
ny_bb = url_to_string('https://www.yelp.com/biz/'+biz_id)
# article = nlp(ny_bb)
# len(article.ents)

In [19]:
#ny_bb

In [41]:
res = requests.get('https://www.yelp.com/biz/'+biz_id)
html =res.text
soup = BeautifulSoup(html, 'html5lib')


In [42]:
#soup

In [46]:
import bs4
res.raise_for_status()
recipeSoup = bs4.BeautifulSoup(res.text, "html.parser")
type(recipeSoup)
#instructions = recipeSoup.find("spnan",_class="lemon--span__373c0__3997G priceRange__373c0__SiD4O", itemprop="priceRange")
#soup = BeautifulSoup(html,"html.parser")

mydivs = soup.findAll("div",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA text-size--large__373c0__1568g"})

mydivs

[]

In [34]:
res.text

'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->\n<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!--> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="no-js" lang="en"> <!--<![endif]-->\n    <head>\n        <script>\n            (function() {\n                var main = null;\n\n                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name==="CSRFallbackError");if(